In [6]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get stock info
msft.info

# get historical market data
hist = msft.history(period="max")

# show actions (dividends, splits)
msft.actions

# show dividends
msft.dividends

# show splits
msft.splits

# show financials
msft.financials
msft.quarterly_financials

# show major holders
msft.major_holders

# show institutional holders
msft.institutional_holders

# show balance heet
msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
msft.cashflow
msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft.sustainability

# show analysts recommendations
msft.recommendations

# show next event (earnings, etc)
msft.calendar

# show ISIN code - *experimental*
# ISIN = International Securities Identification Number
msft.isin

# show options expirations
msft.options

# get option chain for specific expiration
opt = msft.option_chain('2020-03-13')
# data available via: opt.calls, opt.puts

In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from pandas_datareader import data as pdr
yf.pdr_override() # <== that's all it takes :-)

sp500_history = pd.read_csv('./sp500_history.csv')
sp500_history['date'] = pd.to_datetime(sp500_history['date']) 
sp500_history = sp500_history[['date', 'value', 'variable']]

start_date = datetime.strptime('2019-1-1', '%Y-%m-%d')
end_date = datetime.strptime('2019-12-31', '%Y-%m-%d')

# greater than the start date and smaller than the end date
mask = (sp500_history['date'] > start_date) & (sp500_history['date'] <= end_date)
sp500_history = sp500_history.loc[mask]

# Sort
sp500_history.sort_values(['date', 'variable'], ascending=[True, True])

,Unnamed: 0,cik,date,name,value,variable
0,427,1757898.0,11/30/1983,STERIS plc,STE,added_ticker
1,403,315213.0,11/30/1983,Robert Half International,RHI,added_ticker
2,45,315293.0,11/30/1983,Aon plc,AON,added_ticker
3,101,316709.0,11/30/1983,Charles Schwab Corporation,SCHW,added_ticker
4,40,318154.0,11/30/1983,Amgen Inc.,AMGN,added_ticker


In [26]:
columns_to_create = ['total', 'mean', 'std', 'up', 'count', 'max', 'min', 'sharpe', 'uppct']

for column in columns_to_create:
    sp500_history[column] = 0

In [56]:
days_to_subtract = 5
last_date = sp500_history.iloc[0, sp500_history.columns.get_loc('date')]

# for idx, row in enumerate(sp500_history):
row = sp500_history.iloc[0]

# End date is exclusive, so need to increase by 1
prices = pdr.get_data_yahoo(row['value'], start=row['date'] - timedelta(days=days_to_subtract), end=row['date'] + timedelta(days=1))

# Find the previous trading day
prev_date = row['date'] - timedelta(days=1)
while not prev_date in prices.index:
    prev_date -= timedelta(days=1)
prev_close = prices.loc[prev_date]['Close']

# Find the next opening price
next_date = row['date']
while not next_date in prices.index:
    next_date += timedelta(days=1)
next_open = prices.loc[next_date]['Open']

price_change = next_open - prev_close
# row['total'] += price_change
print(prices.head())
#     sp500_history['']


[*********************100%***********************]  1 of 1 completed
                 Open       High        Low      Close  Adj Close    Volume
Date                                                                       
2018-12-27  82.820000  83.739998  80.919998  83.669998  82.914818    720800
2018-12-28  87.930000  89.660004  87.510002  87.940002  87.146286   6518400
2018-12-31  86.760002  87.669998  85.169998  86.900002  86.115669  29983500
2019-01-02  86.540001  87.510002  86.150002  87.500000  86.710258   3119500


In [49]:
prices.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-27,82.820000,83.739998,80.919998,83.669998,82.914818,720800
2018-12-28,87.930000,89.660004,87.510002,87.940002,87.146286,6518400
2018-12-31,86.760002,87.669998,85.169998,86.900002,86.115669,29983500
2019-01-02,86.540001,87.510002,86.150002,87.500000,86.710258,3119500


In [57]:
price_change

-0.3600006103515625

In [58]:
next_open

86.54000091552734